In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Read the CSV file
df = pd.read_csv('dataset.csv')

# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Sentence,Type,Factual/Subjective,Sentiment
0,I'm delighted by the thoughtful gesture.,Affirmation,Subjective,Happiness
1,Rude people are annoying.,Affirmation,Subjective,Anger
2,A cozy blanket makes me happy.,Affirmation,Subjective,Happiness
3,Unseen efforts discourage me.,Affirmation,Subjective,Sadness
4,She doesn’t trust strangers easily.,Negation,Subjective,Neutral


In [2]:
# Verify Class balance "Type" column
df['Type'].value_counts()  

Affirmation    1435
Negation       1014
Name: Type, dtype: int64

In [3]:
# Verify Class balance "Factual/Subjective" column
df['Factual/Subjective'].value_counts()  

Subjective    1457
Factual        992
Name: Factual/Subjective, dtype: int64

In [4]:
# Verify Class balance "Sentiment" column
df['Sentiment'].value_counts()  

Neutral      830
Anger        465
Sadness      442
Happiness    392
Euphoria     320
Name: Sentiment, dtype: int64

In [5]:
# Check for missing values
df.isnull().sum()

Sentence              0
Type                  0
Factual/Subjective    0
Sentiment             0
dtype: int64

In [6]:
# Verify Class balance "Type" column
print(df['Type'].value_counts())
print("-----------------------------")
# Verify Class balance "Factual/Subjective" column
print(df['Factual/Subjective'].value_counts())
print("-----------------------------")
# Verify Class balance "Sentiment" column
print(df['Sentiment'].value_counts())
print("-----------------------------")

Affirmation    1435
Negation       1014
Name: Type, dtype: int64
-----------------------------
Subjective    1457
Factual        992
Name: Factual/Subjective, dtype: int64
-----------------------------
Neutral      830
Anger        465
Sadness      442
Happiness    392
Euphoria     320
Name: Sentiment, dtype: int64
-----------------------------


In [ ]:
# verify and print all repeated values in the column Sentence
print(df['Sentence'].value_counts())

I don't like being ignored.                                  5
I don't like being rushed.                                   5
I don't like being misunderstood.                            4
Gravity pulls objects toward the Earth.                      4
I don't think this is a good idea.                           4
                                                            ..
Earth has 8.7 million species.                               1
I didn’t finish my homework on time.                         1
I’m not interested in politics.                              1
Kangaroos can't walk backwards.                              1
The human impact on the environment is a serious concern.    1
Name: Sentence, Length: 2094, dtype: int64


In [12]:
# Drop duplicates
df = df.drop_duplicates(subset='Sentence')

In [13]:
df

,Sentence,Type,Factual/Subjective,Sentiment
0,I'm delighted by the thoughtful gesture.,Affirmation,Subjective,Happiness
1,Rude people are annoying.,Affirmation,Subjective,Anger
2,A cozy blanket makes me happy.,Affirmation,Subjective,Happiness
3,Unseen efforts discourage me.,Affirmation,Subjective,Sadness
4,She doesn’t trust strangers easily.,Negation,Subjective,Neutral
...,...,...,...,...
2440,He doesn’t share his food.,Negation,Subjective,Neutral
2442,The game didn’t load correctly.,Negation,Factual,Neutral
2444,That rude comment really ticked me off.,Affirmation,Subjective,Anger
2445,The feeling of watching the aurora borealis da...,Affirmation,Subjective,Euphoria


In [14]:
# Verify Class balance "Type" column
print(df['Type'].value_counts())
print("-----------------------------")
# Verify Class balance "Factual/Subjective" column
print(df['Factual/Subjective'].value_counts())
print("-----------------------------")
# Verify Class balance "Sentiment" column
print(df['Sentiment'].value_counts())
print("-----------------------------")

Affirmation    1271
Negation        823
Name: Type, dtype: int64
-----------------------------
Subjective    1253
Factual        841
Name: Factual/Subjective, dtype: int64
-----------------------------
Neutral      719
Anger        378
Sadness      362
Happiness    339
Euphoria     296
Name: Sentiment, dtype: int64
-----------------------------


In [ ]:
# implement textblob library to calculate the polarity of each sentence
from textblob import TextBlob
df['polarity'] = df['Sentence'].apply(lambda x: TextBlob(x).sentiment.polarity)
df.head()


C:\Users\eel20\AppData\Local\Temp\ipykernel_22468\2443365488.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity'] = df['Sentence'].apply(lambda x: TextBlob(x).sentiment.polarity)


,Sentence,Type,Factual/Subjective,Sentiment,polarity
0,I'm delighted by the thoughtful gesture.,Affirmation,Subjective,Happiness,0.550000
1,Rude people are annoying.,Affirmation,Subjective,Anger,-0.550000
2,A cozy blanket makes me happy.,Affirmation,Subjective,Happiness,0.300000
3,Unseen efforts discourage me.,Affirmation,Subjective,Sadness,0.000000
4,She doesn’t trust strangers easily.,Negation,Subjective,Neutral,0.433333


In [17]:
# textblob library to calculate the subjectivity of each sentence
df['subjectivity'] = df['Sentence'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df.head()


C:\Users\eel20\AppData\Local\Temp\ipykernel_22468\1240920775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subjectivity'] = df['Sentence'].apply(lambda x: TextBlob(x).sentiment.subjectivity)


,Sentence,Type,Factual/Subjective,Sentiment,polarity,subjectivity
0,I'm delighted by the thoughtful gesture.,Affirmation,Subjective,Happiness,0.550000,0.600000
1,Rude people are annoying.,Affirmation,Subjective,Anger,-0.550000,0.750000
2,A cozy blanket makes me happy.,Affirmation,Subjective,Happiness,0.300000,0.875000
3,Unseen efforts discourage me.,Affirmation,Subjective,Sadness,0.000000,0.000000
4,She doesn’t trust strangers easily.,Negation,Subjective,Neutral,0.433333,0.833333
